In [ ]:
import cv2
import numpy as np
import pytesseract
import os
import re
import pandas as pd

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
import matplotlib.pylab as plt
import tensorflow as tf

folder = r"../../../1.DATA/3.forms_images/Registry2013/"
year = '2013'
forms_list = os.listdir(folder)

# print(forms_list)
#  Load the model
from tensorflow.keras.models import load_model
model = load_model('checkboxes_classifier.h5')

roi =[[(482,994),(495,1007),'box', 'RemediationMeasures.NoRemediationMeasures'],
      [(482,977),(495,990), 'box', 'RemediationMeasures.Others'],
      [(251,977),(263,990), 'box', 'RemediationMeasures.ContaminatedSoil/WasteCover'],
      [(47,977),(60,990), 'box', 'RemediationMeasures.ReductiveDichlorination'],
      [(482,957),(495,970), 'box', 'RemediationMeasures.MonitoredNaturalAttenuation'],
      [(251,957),(263,970),'box', 'RemediationMeasures.GeotechnicalEncapsulation'],
      [(47,957),(60,970), 'box', 'RemediationMeasures.MultiphaseExtraction'],
      [(482,939),(495,951), 'box', 'RemediationMeasures.Biopile'],
      [(251,939),(263,951),'box', 'RemediationMeasures.FreePhaseRecovery'],
      [(47,939),(60,951),'box', 'RemediationMeasures.Bioventing'],
    [(251,922),(263,934),'box', 'RemediationMeasures.RemovalOfSoil/Residues'],
    [(483,919),(496,932),'box', 'RemediationMeasures.Phytoremediation'],
    [(47,918),(60,931),'box', 'RemediationMeasures.Biosparging'],
    [(482,902),(495,915),'box', 'RemediationMeasures.Bioremediation'],
    [(251,902),(263,915),'box', 'RemediationMeasures.SoilWashing'],
    [(47,902),(60,915), 'box', 'RemediationMeasures.AirSparging'],
    [(482,884),(495,896),'box', 'RemediationMeasures.Hydraulicbarrier'],
    [(251,884),(263,896), 'box', 'RemediationMeasures.ReactiveBarrier'],
    [(47,884),(60,896),'box', 'RemediationMeasures.SoilVapourExtraction'],
    [(482,864),(495,877),'box', 'RemediationMeasures.PhysicalBarrier'],
    [(251,864),(263,877),'box', 'RemediationMeasures.ChemicalOxidation/Reduction'],
    [(47,864),(60,877), 'box', 'RemediationMeasures.Pump&Treat'],
    
    [(690,830),(703,842),'box','InstitutionalControlMeasures.ConstructionWorkers.Implemented'],
    [(615,830),(627,842),'box', 'InstitutionalControlMeasures.ConstructionWorkers.CommunicatedToEA'],
    [(513,830),(526,842),'box', 'InstitutionalControlMeasures.ConstructionWorkers.RiskAssessment|InterventionPlan'],
    [(690,812),(703,824),'box', 'InstitutionalControlMeasures.BuildingsUse.Implemented'],
    [(613,812),(626,824),'box', 'InstitutionalControlMeasures.BuildingsUse.CommunicatedToEA'],
    [(513,812),(526,824),'box', 'InstitutionalControlMeasures.BuildingsUse.RiskAssessment|InterventionPlan'],
    [(690,794),(703,806), 'box', 'InstitutionalControlMeasures.FoodComsumption.Implemented'],
    [(615,796),(627,808),'box','InstitutionalControlMeasures.FoodComsumption.CommunicatedToEA'],
    [(513,796),(526,808),'box', 'InstitutionalControlMeasures.FoodComsumption.RiskAssessment|InterventionPlan'],
    [(690,776),(703,788),'box', 'InstitutionalControlMeasures.SurfaceWaterUse.Implemented'],
    [(615,777),(627,790),'box', 'InstitutionalControlMeasures.SurfaceWaterUse.CommunicatedToEA'],
    [(513,777),(526,790),'box', 'InstitutionalControlMeasures.SurfaceWaterUse.RiskAssessment|InterventionPlan'],
    [(690,758),(703,770),'box', 'InstitutionalControlMeasures.GroundwaterUse.Implemented'],
    [(615,758),(627,771),'box', 'InstitutionalControlMeasures.GroundwaterUse.CommunicatedToEA'],
    [(513,758),(526,771),'box', 'InstitutionalControlMeasures.GroundwaterUse.RiskAssessment|InterventionPlan'],
    [(690,738),(703,752), 'box', 'InstitutionalControlMeasures.SoilUse.Implemented'],
    [(615,741),(627,753), 'box', 'InstitutionalControlMeasures.SoilUse.CommunicatedToEA'],
    [(513,741),(526,753), 'box', 'InstitutionalControlMeasures.SoilUse.RiskAssessment|InterventionPlan'],
    
    [(60,829),(73,841), 'box', 'EmergencyMeasures.BanOnFoodConsumption'],
    [(60,811),(73,824),'box', 'EmergencyMeasures.ExcavationBan'],
    [(60,796),(73,808),'box', 'EmergencyMeasures.BuildingsBan'],
    [(60,779),(73,791),'box', 'EmergencyMeasures.Closing/InterdictionOfSupplyWells'],
    [(60,761),(73,774), 'box', 'EmergencyMeasures.RemovalOfmaterials'],
    [(60,746),(73,758),'box', 'EmergencyMeasures.EnvironmentalMonitoring'],
    [(60,729),(73,741),'box', 'EmergencyMeasures.MonitoringTheExplosivenessIndex'],
    [(60,711),(73,724),'box', 'EmergencyMeasures.Ventilation/ExhaustionOfConfinedSpaces'],
    [(60,696),(73,708), 'box', 'EmergencyMeasures.IsolationOfTheArea'],
    
    [(44,654),(56,667),'box', 'Contaminants.POPs'],
    [(44,636),(56,649), 'box', 'Contaminants.FreePhase'],
    [(645,636),(657,648),'box', 'Contaminants.Others'],
    [(646,615),(658,628), 'box', 'Contaminants.TPH'],
    [(645,591),(657,604), 'box', 'Contaminants.Microbiological'],
    [(645,569),(657,581), 'box', 'Contaminants.Radionuclides'],
    [(645,544),(657,557),'box', 'Contaminants.Anilines'],
    [(645,520),(657,533),'box', 'Contaminants.Dioxins&Furans'],
    [(645,498),(657,510),'box', 'Contaminants.Phthalates'],
    [(645,473),(657,486), 'box', 'Contaminants.Biocides'],
    [(645,451),(657,463),'box', 'Contaminants.Fenols'],
    
    
    
    [(394,638),(406,650),'box', 'Contaminants.Methane/OtherGazes'],
    [(394,615),(406,628),'box', 'Contaminants.PCBs'],
    [(394,593),(406,605),'box', 'Contaminants.PAHs'],
    [(394,569),(406,581),'box', 'Contaminants.HalogenatedAromaticSolvents'],
    [(394,544),(406,557),'box', 'Contaminants.AromaticSolvents'],
    [(394,522),(406,534),'box', 'Contaminants.HalogenatedSolvents'],
    [(394,497),(406,510),'box', 'Contaminants.OtherInorganic'],
    [(394,473),(406,486),'box', 'Contaminants.Metals'],
    [(394,451),(406,463),'box', 'Contaminants.AutomotiveFuels'],
    
    [(289,602),(301,614),'box', 'ImpactedMedia.Biota.Outside'],
    [(207,602),(219,614),'box', 'ImpactedMedia.Biota.Inside'],
    [(289,583),(301,595),'box', 'ImpactedMedia.Air.Outside'],
    [(207,583),(219,595),'box', 'ImpactedMedia.Air.Inside'],
    [(287,565),(300,578),'box', 'ImpactedMedia.Sediments.Outside'],
    [(206,565),(218,578),'box', 'ImpactedMedia.Sediments.Inside'],
    [(287,546),(300,559),'box', 'ImpactedMedia.Groundwater.Outside'],
    [(206,546),(218,559),'box', 'ImpactedMedia.Groundwater.Inside'],
    [(287,530),(300,542),'box', 'ImpactedMedia.SurfaceWater.Outside'],
    [(206,530),(218,542),'box', 'ImpactedMedia.SurfaceWater.Inside'],
    [(287,513),(300,525),'box', 'ImpactedMedia.Soil.Outside'],
    [(206,513),(218,525),'box', 'ImpactedMedia.Soil.Inside'],
    [(287,493),(300,507),'box', 'ImpactedMedia.SuperficialSoil.Outside'],
    [(206,493),(218,507),'box', 'ImpactedMedia.SuperficialSoil.Inside'],
    
    
    [(287,408),(300,421),'box', 'ContaminationSource.Accidents'],
    [(39,407),(52,419),'box', 'ContaminationSource.Disposal'],
    [(416,405),(429,418),'box', 'ContaminationSource.Unknown'],
    [(175,405),(187,418),'box','ContaminationSource.Infiltration'],
    [(588,386),(600,399),'box', 'ContaminationSource.WastewaterTreatment'],
    [(415,386),(428,399),'box','ContaminationSource.AtmosphericEmissions'],
    [(289,386),(301,399),'box', 'ContaminationSource.Maintenance'],
    [(173,386),(186,399),'box', 'ContaminationSource.Production'],
    [(39,386),(52,399),'box', 'ContaminationSource.Storage'],
    
    [(390,339),(402,352),'box', 'ManagementPhase.ClosingMonitoring1'],
    [(390,321),(402,333),'box', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring1'],
    [(390,300),(402,313),'box', 'ManagementPhase.RemediationProject'],
    [(390,282),(402,294),'box', 'ManagementPhase.InterventionPlan'],
    [(390,262),(402,274),'box', 'ManagementPhase.RiskAssessment/RiskManagement'],
    [(390,244),(402,257),'box', 'ManagementPhase.DetailedInvestigation'],
    [(390,223),(402,235),'box','ManagementPhase.ConfirmatoryInvestigation1'],
    [(390,203),(402,215),'box', 'ManagementPhase.PreliminaryAssessment'],
    
    [(55,316),(68,329),'box', 'ManagementPhase.ClosingMonitoring'],
    [(55,294),(68,306),'box', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring'],
    [(55,274),(68,286),'box', 'ManagementPhase.DetailedInvestigation&InterventionPlan'],
    [(55,249),(68,262),'box', 'ManagementPhase.ConfirmatoryInvestigation'],
    [(55,227),(68,239),'box', 'ManagementPhase.LeakRemovalMeasures'],
    [(55,203),(68,215), 'box', 'ManagementPhase.OccurenceAssessment'],
    
    [(618,161),(631,174), 'box', 'Reuse'],
    [(667,126),(679,137),'box', 'Activity.Unknown'],
    [(588,123),(599,135),'box', 'Activity.Agriculture'],
    [(493,123),(505,135),'box', 'Activity.Accidents'],
    [(414,123),(426,135), 'box', 'Activity.Residue'],
    [(280,123),(291,135),'box', 'Activity.Gas_Station'], 
    [(202,123),(213,135),'box', 'Activity.Commerce'],
    [(115,123),(127,135),'box', 'Activity.Industry']]

data_boxes = []
pixels_check= []
value = 0

img = cv2.imread(folder+'/'+forms_list[0])
img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
img = cv2.resize(img,(0,0),fx=1,fy=1)
#forms_list = ['CETESB_registry_2019-1.jpg', 'CETESB_registry_2019-10.jpg',
#              'CETESB_registry_2019-100.jpg', 'CETESB_registry_2019-1000.jpg', 'CETESB_registry_2019-1001.jpg']
for i,r in enumerate(roi):
    x=r[0][0]
    y=r[0][1]
    cv2.rectangle(img,(r[0][0],r[0][1]),(r[1][0],r[1][1]),(0,255,0),2)
    cv2.putText(img=img, text=r[3],org =(x-2,y-2), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.25, color=(0, 0, 0),thickness=1)
    
cv2.imwrite(f'Detected_Textboxes_{year}_updated.jpg',img)
for j,y in enumerate(forms_list):
    data = []
    pixels = []
    img = cv2.imread(folder+"/"+ y)
    s_img = cv2.resize(img, (0, 0), fx=1,fy=1)
#     cv2.imwrite()
#     cv2.imshow(y,s_img)
#     cv2.waitKey(0)


    print(f'######### Extracting Data from {j} ##########')
    regex = re.compile(r'\d+')
    form_nb = regex.findall(y)[1]
    data.append(year)
    data.append(form_nb)
#     pixels.append(form_nb)
    for x,r in enumerate(roi):
        
        imgCrop = s_img[r[0][1]:r[1][1],r[0][0]:r[1][0]]
#         gray = cv2.cvtColor(imgCrop,cv2.COLOR_BGR2GRAY)

#         n_white_pix = np.sum(gray == 255)

#         pixels.append(n_white_pix)
        imgCrop_resized = cv2.resize(imgCrop, (112,112))

        X = image.img_to_array(imgCrop_resized)
        X = np.expand_dims(X,axis=0)
        images = np.vstack([X])
        val = model.predict(images)
        if val == 0:
            ### box is checked ###
            value = 1
            data.append(value)
        else: 
            ### box is not checked ###
            value = 0
            data.append(value)
#     print(data)


#     pixels_check.append(pixels)
    data_boxes.append(data)

list_columns = [r[3] for r in (roi)]
list_columns.insert(0,'FormNb')
list_columns.insert(0,'Year')
# # Create the pandas DataFrame
# columns = ['FormNb', 'Activity.Industry', 'Activity.Commerce', 'Activity.Gas_Station', 'Activity.Residue', 'Activity.Accidents', 
#            'Activity.Agriculture', 'Activity.Unknown', 'Reuse', 'ManagementPhase.OccurenceAssessment', 
#            'ManagementPhase.LeakRemovalMeasures', 'ManagementPhase.ConfirmatoryInvestigation', 
#            'ManagementPhase.DetailedInvestigation&InterventionPlan', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring', 
#            'ManagementPhase.ClosingMonitoring', 'ManagementPhase.PreliminaryAssessment', 'ManagementPhase.ConfirmatoryInvestigation1',
#            'ManagementPhase.DetailedInvestigation', 'ManagementPhase.RiskAssessment', 'ManagementPhase.InterventionPlan', 
#            'ManagementPhase.RemediationProject', 'ManagementPhase.RemediationWithEfficiency&EfficacyMonitoring1', 
#            'ManagementPhase.ClosingMonitoring1', 'ContaminationSource.Storage', 'ContaminationSource.Disposal', 
#            'ContaminationSource.Production', 'ContaminationSource.Infiltration', 'ContaminationSource.Maintenance', 
#            'ContaminationSource.Accidents', 'ContaminationSource.AtmosphericEmissions', 'ContaminationSource.Unknown', 
#            'ContaminationSource.WastewaterTreatment', 'ImpactedMedia.SuperficialSoil.Inside', 'ImpactedMedia.SuperficialSoil.Outside', 
#            'ImpactedMedia.Soil.Inside', 'ImpactedMedia.Soil.Outside', 'ImpactedMedia.SurfaceWater.Inside',
#            'ImpactedMedia.SurfaceWater.Outside', 'ImpactedMedia.Groundwater.Inside', 'ImpactedMedia.Groundwater.Outside', 
#            'ImpactedMedia.Sediments.Inside', 'ImpactedMedia.Sediments.Outside', 'ImpactedMedia.Air.Inside', 
#            'ImpactedMedia.Air.Outside', 'ImpactedMedia.Biota.Inside', 'ImpactedMedia.Biota.Outside', 'Contaminants.Metals',
#            'Contaminants.OtherInorganic', 'Contaminants.HalogenatedSolvents', 'Contaminants.AromaticSolvents', 
#            'Contaminants.HalogenatedAromaticSolvents', 'Contaminants.PAHs', 'Contaminants.PCBs', 'Contaminants.Methane',
#            'Contaminants.AutomotiveFuels', 'Contaminants.OtherVapours/Gazes', 'Contaminants.Fenols', 'Contaminants.Biocides',
#            'Contaminants.Phthalates', 'Contaminants.Dioxins&Furans', 'Contaminants.Anilines', 'Contaminants.Radionuclides', 
#            'Contaminants.Microbiological', 'Contaminants.TPH', 'Contaminants.Others', 'Contaminants.FreePhase', 'Contaminants.POPs',
#            'EmergencyMeasures.IsolationOfTheArea', 'EmergencyMeasures.Ventilation/ExhaustionOfConfinedSpaces',
#            'EmergencyMeasures.MonitoringTheExplosivenessIndex', 'EmergencyMeasures.EnvironmentalMonitoring', 
#            'EmergencyMeasures.RemovalOfmaterials', 'EmergencyMeasures.Closing/InterdictionOfSupplyWells', 
#            'EmergencyMeasures.BuildingsBan', 'EmergencyMeasures.ExcavationBan', 'EmergencyMeasures.BanOnFoodConsumption',
#            'InstitutionalControlMeasures.SoilUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.SoilUse.CommunicatedToEA', 'InstitutionalControlMeasures.SoilUse.Implemented',
#            'InstitutionalControlMeasures.GroundwaterUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.GroundwaterUse.CommunicatedToEA', 'InstitutionalControlMeasures.GroundwaterUse.Implemented', 
#            'InstitutionalControlMeasures.SurfaceWaterUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.SurfaceWaterUse.CommunicatedToEA', 'InstitutionalControlMeasures.SurfaceWaterUse.Implemented',
#            'InstitutionalControlMeasures.FoodConsumption.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.FoodConsumption.CommunicatedToEA', 'InstitutionalControlMeasures.FoodConsumption.Implemented', 
#            'InstitutionalControlMeasures.BuildingsUse.RiskAssessment|InterventionPlan', 
#            'InstitutionalControlMeasures.BuildingsUse.CommunicatedToEA', 'InstitutionalControlMeasures.BuildingsUse.Implemented',
#            'RemediationMeasures.Pump&Treat', 'RemediationMeasures.SoilVapourExtraction', 'RemediationMeasures.AirSparging', 
#            'RemediationMeasures.Biosparging', 'RemediationMeasures.Bioventing', 'RemediationMeasures.MultiphaseExtraction', 
#            'RemediationMeasures.ReductiveDichlorination', 'RemediationMeasures.InSituHeatTreatment', 'RemediationMeasures.ChemicalOxidation', 'RemediationMeasures.ChemicalReduction', 'RemediationMeasures.ReactiveBarrier', 'RemediationMeasures.SoilWashing', 'RemediationMeasures.RemovalOfSoil/Residues', 'RemediationMeasures.FreePhaseRecovery', 'RemediationMeasures.GeotechnicalEncapsulation', 'RemediationMeasures.ContaminatedSoil/WasteCover', 'RemediationMeasures.PhysicalBarrier', 'RemediationMeasures.Hydraulicbarrier', 'RemediationMeasures.Bioremediation', 'RemediationMeasures.Phytoremediation', 'RemediationMeasures.Biopile', 'RemediationMeasures.MonitoredNaturalAttenuation', 'RemediationMeasures.Others', 'RemediationMeasures.NoRemediationMeasures', 'EngineeringControlMeasures.ProjectSuitability', 'EngineeringControlMeasures.Waterproofing', 
#            'EngineeringControlMeasures.Paving', 'EngineeringControlMeasures.Others']
# pixel_boxes = pd.DataFrame(pixels_check, columns=list_columns)
data_boxes = pd.DataFrame(data_boxes, columns=list_columns)

# pixel_boxes.to_excel(f'../../3.Output/dec2013/Pixel_box_trial_{year}_updated.xlsx')
data_boxes.to_excel(f'../Final_Boxes/{year}_model_classifier.xlsx')


######### Extracting Data from 0 ##########
1/1 [==============================] - 1s 1s/step
######### Extracting Data from 1 ##########
1/1 [==============================] - 1s 896ms/step


1/1 [==============================] - 1s 820ms/step
######### Extracting Data from 2 ##########
1/1 [==============================] - 1s 586ms/step


1/1 [==============================] - 1s 665ms/step
######### Extracting Data from 3 ##########
1/1 [==============================] - 0s 351ms/step
######### Extracting Data from 4 ##########
1/1 [==============================] - 0s 239ms/step


1/1 [==============================] - 1s 626ms/step
######### Extracting Data from 5 ##########
1/1 [==============================] - 1s 650ms/step


1/1 [==============================] - 0s 373ms/step
######### Extracting Data from 6 ##########
1/1 [==============================] - 0s 383ms/step


...

1/1 [==============================] - 0s 440ms/step
######### Extracting Data from 22 ##########
1/1 [==============================] - 0s 481ms/step


1/1 [==============================] - 0s 450ms/step
######### Extracting Data from 23 ##########
1/1 [==============================] - 0s 485ms/step
######### Extracting Data from 24 ##########
1/1 [==============================] - 1s 622ms/step


1/1 [==============================] - 1s 580ms/step
######### Extracting Data from 25 ##########
1/1 [==============================] - 0s 361ms/step


1/1 [==============================] - 0s 453ms/step
######### Extracting Data from 26 ##########
1/1 [==============================] - 1s 615ms/step


1/1 [==============================] - 0s 452ms/step
######### Extracting Data from 27 ##########
1/1 [==============================] - 1s 531ms/step
